# Kas süntaks on konserveerunud 10 000 juh advmod lauset

Loeme failist 10 000 juhuslikku lauset

In [1]:
import csv
import pandas as pd
import json

Loeme laused failist. 

In [2]:
sentencesTSV = 'result/1_sentences_with_advmod.tsv'

df = pd.read_csv(sentencesTSV, sep='\t', header=None, quoting=csv.QUOTE_NONE)  
df.shape

(10626918, 3)

In [3]:
df.head()

,0,1,2
0,0,69,"Endine punkar , tulevane lendur , karateka ja ..."
1,0,262,"Bändi , mille solist ei osanud laulda , kidram..."
2,0,739,Kui sageli tuli sul kooli juhtkonna ees aru an...
3,0,791,"Mind kutsuti tihti direktori jutule , aga mitt..."
4,0,1080,"Tol ajal oli mu ainuke prioriteet muusika , mi..."


Ajame read juhuslikku järjekorda

In [4]:
def shuffle_rows(df):
    return df.sample(frac=1).reset_index(drop=True)

df = shuffle_rows(df)
df.head()

,0,1,2
0,690671,1338,Lavalt tuli ainult kulda .
1,655256,2286,Kuskilt live-feed ka tuleb ?
2,640616,1937228,Kui nii näis siis palun ka avalikult vabandust .
3,700938,303798,võibolla on mul tõesti midagi kahe silma vahel...
4,405340,2920,"Ööpoed müüvad öösiti vaid konserve ja vorsti ,..."


### 10 000 esimest juhuslikku lauset

Laseme süntaksi peale
Eemaldame admvod koos selle alampuuga
Kontrollime, kas süntaks on konserveerunud

In [5]:
import stanza
import os
from estnltk import Text
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger
from estnltk.converters.conll_exporter import  sentence_to_conll
import numpy as np


In [6]:
model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
#stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, resources_path=model_path)
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


In [7]:
def write_to_file(t, f):
    f.write(t)

def get_children(span, lst):
    if not span.children:
        return
    else:
        for child in span.children:
            lst += [int(child.id)-1]
            get_children(child, lst)


def remove_deprel(sentence, dprl, stanza_tagger):
    changes = {}
    no_rem_dprls = 0
   
    #Lausete lühendamine
    sent = sentence.sentences.text
    len_sent = len(sent)
    lsts = []
    for span in sentence.stanza_syntax:
        if span.deprel == dprl:
            lst = [int(span.id)-1]
            get_children(span, lst)
            lsts += lst

    sent = np.array(sent)
    sent[list(set(lsts))] = ":::"
    sent = list(sent)
    sent = list(filter(lambda a: a != ":::", sent))
    if  len(sent) != len_sent:
       
        no_rem_dprls += (len_sent - len(sent))
    
    short_sent = " ".join(sent)
    short_sent = short_sent[0].upper() + short_sent[1:]
    short_sent_txt = Text(short_sent)
    short_sent_txt.analyse('all')
    stanza_tagger.tag(short_sent_txt)
    len_short_sent = len(sent)
    
    pikkus = 0
    e = 0
    m = 0

    sent_deprel = ''
    changed = False
    while len_sent > pikkus:
        sp_long = sentence.stanza_syntax[pikkus]
        if  e < len_short_sent:
            sp_short = short_sent_txt.stanza_syntax[e]
        
        if sp_short.text.lower() == sp_long.text.lower() and pikkus == e + m and e < len_short_sent:
            sent_deprel += sp_short["deprel"] + " "
            if sp_short["deprel"] != sp_long.deprel:
                changed = True
                key = sp_long.deprel + "_" + sp_short["deprel"]
                if key not in changes.keys():
                    changes[key] = 1
                else:
                    changes[key] += 1
            e += 1
        else:
            sent_deprel += "_" + " "
            m += 1
        pikkus += 1     
    
    # changed - kas muutus
    # short_sent_txt.stanza_syntax
    # no_rem_dprls, changes
    return {'changed': changed \
                , 'short_sent_txt': short_sent_txt \
                , 'no_rem_dprls': no_rem_dprls \
                , 'changes': changes \
                , 'sent_deprel': sent_deprel}
   


Leiame 10 korda statistika 10000 juhusliku lause kohta, milles esines deprel *advmod*.
    * mitmes lauses süntaks konserveerub,
    * mitmes lauses süntaks ei konserveeru. 

Salvestame need läbivaadatud laused failidesse, et saaks vajadusel kontrollida.

In [8]:
%%time

# deprel to remove
dprl = "advmod"

RESULTS_PATH = "./result/"


count = 10000
iterations = 2


stats = {'iteration': []
             , 'total_sentences': []
             , 'syntax_changed': []
             , 'syntax_conserved': []
             , 'allchanges': []
        }
for i in range(iterations):    
    iteration = i+1
    df = shuffle_rows(df)
    print (f"Iteratsioon {iteration}")
    sentencesFilename = f"{RESULTS_PATH}2_sentences_{dprl}_set_{iteration}.tsv"
   
    sentencesF = open(sentencesFilename, 'w', encoding='utf-8')
   
    # create the csv writer
    wSentences = csv.writer(sentencesF, delimiter='\t')
  


    wSentences.writerow(('collectionID', 'sentenceStart', 'changed', 'changes', 'original', 'short',))
    
    proc_txt_no = 0 # counter for texts
    no_rem_dprlss = 0   # count number of removed deprels
    no_changed_syntaxs = 0   # count sentences where syntax changed

    sentences_count = 0
    syntax_conserved = 0
    syntax_changed = 0

    all_changes = {}

    for ind in range(df.shape[0]):
        #ignoreerime lauseid, kus pole advmod'i siiski sees

        proc_txt_no = df[0][ind]
        sent_span_start = df[1][ind]

        txt = df[2][ind]
        #print (txt)
        sent = stanza_tagger.tag(Text(txt).analyse("all"))

        # siia kontroll, et advmod ikka sees
        if not dprl in sent.deprel: continue

        sentences_count += 1
        result = remove_deprel(sent, dprl, stanza_tagger)
        if not len(result['changes']):
            syntax_conserved +=1
        else:
            syntax_changed +=1
        
        all_changes = {k: all_changes.get(k, 0) + result['changes'].get(k, 0) for k in set(all_changes) | set(result['changes'])}
       
        wSentences.writerow( [str(proc_txt_no) \
                , str(sent_span_start) \
                , str(int(result['changed'] == True)) \
                , json.dumps(result['changes']) \
                , sent.text \
                , result['short_sent_txt'].text  ], )
       

        if sentences_count >= count: break 
    
    sentencesF.close()

    stats['iteration'].append(iteration)
    stats['syntax_changed'].append(syntax_changed)
    stats['syntax_conserved'].append(syntax_conserved)
    stats['total_sentences'].append(sentences_count)
    stats['allchanges'].append(json.dumps(all_changes))
    
    print (f'Teisendatud {sentences_count} lauset.')
    print (f'Kokku vaadati {ind+1} lauset.')
    print (f'Süntaks konserveerunud {syntax_conserved} lausel.')

    print (all_changes)


  

Iteratsioon 1
Teisendatud 10000 lauset.
Kokku vaadati 10129 lauset.
Süntaks konserveerunud 6540 lausel.
{'det_nmod': 13, 'ccomp_csubj': 1, 'nmod_advcl': 5, 'compound:prt_conj': 3, 'flat_appos': 3, 'acl_obl': 4, 'xcomp_nmod': 4, 'xcomp_acl:relcl': 2, 'csubj:cop_conj': 8, 'parataxis_conj': 18, 'ccomp_case': 1, 'nsubj:cop_advcl': 4, 'root_nsubj': 46, 'obl_xcomp': 17, 'conj_obj': 27, 'nummod_nsubj': 1, 'mark_cc:preconj': 1, 'ccomp_acl': 18, 'cc_mark': 31, 'cc_aux': 1, 'obl_appos': 12, 'advcl_nsubj': 13, 'csubj_conj': 7, 'root_obj': 14, 'obj_appos': 7, 'acl:relcl_nsubj:cop': 2, 'advcl_aux': 6, 'parataxis_csubj': 1, 'obl_nmod': 252, 'csubj_acl:relcl': 2, 'nsubj:cop_flat': 6, 'obl_nsubj:cop': 11, 'aux_advcl': 7, 'xcomp_nsubj': 4, 'nummod_flat': 2, 'csubj:cop_advcl': 1, 'amod_advcl': 5, 'obl_obj': 93, 'acl_advmod': 1, 'root_csubj': 2, 'amod_nsubj:cop': 1, 'conj_xcomp': 31, 'amod_cop': 3, 'conj_det': 1, 'nmod_amod': 5, 'nsubj:cop_root': 49, 'ccomp_xcomp': 2, 'acl:relcl_aux': 2, 'parataxis_disco

In [9]:
dfStat = pd.DataFrame(stats)
dfStat.to_csv(f"{RESULTS_PATH}2_{dprl}_stats.tsv",sep='\t', index=False)

In [10]:
dfChanged = pd.read_csv(sentencesFilename, sep='\t', quotechar = '"')  
dfChanged.shape

(10000, 6)

In [11]:
dfChanged.sample(10)

,collectionID,sentenceStart,changed,changes,original,short
2723,519470,1339,1,"{""root_nmod"": 1, ""nsubj_appos"": 1, ""obj_root"": 1}",Mullu testisid Korjus-Jakk MTH mootoreid ning said võistlusteks tasuta jõuallikaid .,Testisid Korjus-Jakk MTH mootoreid ning said võistlusteks tasuta jõuallikaid .
1466,281940,920,0,{},Ja ometi pole Malèna lõpuni lahtiharutatud .,Ja pole Malèna lõpuni lahtiharutatud .
971,514774,13694,0,{},Tollal oli teatavasti raske mis tahes alkohoolset jooki soetada .,Oli raske alkohoolset jooki soetada .
3387,6336,126,0,{},"Samu andmeid kinnitas ka Ingušimaa president Ruslan Aušev , kelle sõnul peidavad inimesed ennast keldrites ega saa linnast lahkuda , kuna pole mingeid transpordivahendeid .","Samu andmeid kinnitas Ingušimaa president Ruslan Aušev , kelle sõnul peidavad inimesed ennast keldrites ega saa linnast lahkuda , kuna pole mingeid transpordivahendeid ."
3758,615707,1437,0,{},"Eelnevast lähtuvalt üritatakse avalikkusele jätta mulje , et küll me toimetame siin vaikselt ja tagasihoidlikult ning ega me eriti ei kuluta .","Üritatakse avalikkusele jätta mulje , et me toimetame ning me ei kuluta ."
8459,646648,692,0,{},"Kui jätta kõrvale paari üksiku talu tragöödia , ei ole kaugemalt vaadates praegusel piiril häda midagi .","Kui jätta kõrvale paari üksiku talu tragöödia , ei ole vaadates praegusel piiril häda midagi ."
1346,647835,1392,0,{},Mingisugust auto-adjust nuppu ei ole või ?,Mingisugust auto-adjust nuppu ei ole ?
9320,705041,49740,0,{},"Kõik kolm valitsust on väljendanud oma soovi jõustada leping hiljemalt 31. oktoobriks käesoleval aastal , mis eeldaks viimase ratifitseerimisteate saabumist 31. augustil .","Kõik kolm valitsust on väljendanud oma soovi jõustada leping 31. oktoobriks käesoleval aastal , mis eeldaks viimase ratifitseerimisteate saabumist 31. augustil ."
3440,381325,4890,0,{},"Ja veel : äris kehtib reegel , et hulgi on odavam .",": äris kehtib reegel , et on odavam"
9842,681223,1671,1,"{""cc_advmod"": 1, ""nsubj_nmod"": 1, ""nmod_flat"": 1}","ja neljandax iga soliidne yrituse korraldaja ( kui mitte soliidne , siis seda enam on vaja ) mainix ka zyrii koosseisu , mitte nii et lihtsalt mingi 4 inimest .","Ja neljandax iga soliidne yrituse korraldaja ( kui soliidne , siis on vaja ) mainix zyrii koosseisu , mingi 4 inimest ."
